# Machine Learning Tutorial 

The naive Bayes multiclass approach is an extension of the naive Bayes approach. It can be trained to output binary images given an input color image. Unlike the naive Bayes method, the naive Bayes multiclass approach can be trained to classify two or more classes, defined by the user. Additionally, the naive Bayes multiclass method is trained using colors sparsely sampled from images rather than the need to label all pixels in a given image.

To train the classifier, we need to build a table of red, green, and blue color values for pixels sampled evenly from each class. The idea here is to collect a relevant sample of pixel color data for each class. The size of the sample needed to build robust probability density functions for each class will depend on a number of factors, including the variability in class colors and imaging quality/reproducibility. To collect pixel color data we currently use the Pixel Inspection Tool in [ImageJ](https://imagej.nih.gov/ij/). 





plantcv-train.py naive_bayes_multiclass --file pixel_samples.txt --outfile naive_bayes_pdfs.txt --plots

In [ ]:
import cv2
from plantcv import plantcv as pcv

In [ ]:
class options:
    def __init__(self):
        self.image = "img/tutorial_images/machine_learning/color_image.jpg"
        self.debug = "plot"
        self.writeimg= False 
        self.outdir = "."
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug

In [ ]:
# Read image (sometimes you need to run this line twice to see the image) 
img, path, filename = pcv.readimage(args.image)

In [ ]:
# Use the output file from `plantcv-train.py` to run the multiclass 
# naive bayes classification on the image. The function below will 
# print out 4 masks (plant, pustule, chlorosis, background)

# Inputs: 
#   rgb_img - RGB image data 
#   pdf_file - Output file containing PDFs from `plantcv-train.py`
mask = pcv.naive_bayes_classifier(img, pdf_file="img/tutorial_images/machine_learning/machine_learning.txt")

In [ ]:
# We can apply each mask to the original image to more accurately 
# see what got masked

# Inputs:
#   rgb_img - RGB image data 
#   mask - Binary mask image data 
#   mask_color - 'white' or 'black' 
plant_img = pcv.apply_mask(mask=(mask['plant']), rgb_img=img, mask_color='black')
pustule_img = pcv.apply_mask(mask=(mask['pustule']), rgb_img=img, mask_color='black')
chlorosis_img = pcv.apply_mask(mask=(mask['chlorosis']), rgb_img=img, mask_color='black')
background_img = pcv.apply_mask(mask=(mask['background']), rgb_img=img, mask_color='black')

In [ ]:
# To see all of these masks together we can plot them with plant set to green,
# chlorosis set to blue, and pustule set to red.

classified_img = cv2.merge([mask['chlorosis'], mask['plant'], mask['pustule']])
pcv.plot_image(classified_img)